# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [37]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
data = dict()

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [15]:
def movie_title_url_actor(top_num, major_actor_num, dic):
    '''
        top_num: 예매순 상위 top_num개
        major_actor_num: 주연배우 major_actor_num 명
        dic: 딕셔너리
    '''
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    titles = soup.find_all(class_ ='tit')
    for title in titles[0:top_num]:
        dic[title.find('a').text] = dict()
        dic[title.find('a').text]['url'] = title.find('a').get('href')
    
    idx= 0
    infos = soup.find_all(class_ ='info_txt1')
    for info in infos[0:top_num]:
        actors_list = info.select_one("dd:nth-of-type(3)").text.strip().split(',')
        actors = []
        n = 1
        for actor in actors_list:
            if n==(major_actor_num+1):
                break
            actors.append(actor.strip())
            n+=1
        dic[list(dic.keys())[idx]]['actors'] = actors
        idx+=1
        
    return dic

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [6]:
def get_grade(url):
    '''
        url: 위의 함수에서 저장한 url
    '''
    full_url = 'https://movie.naver.com'+url
    res = requests.get(full_url, headers={'User-Agent':'Mozilla/5.0'})
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    p = re.compile('[0-9]{2}.[0-9]{1}') #평점 가져오는 방법들 중 하나 (별점 style의 width로 계산)
    likey = soup.find_all(class_ ='st_on')
    
    # [관람객, 평론가, 네티즌]
    grades = []
    for like in likey[0:3]:
        grades.append(float(p.findall(like.get('style'))[0])*0.1)
        
    return grades

In [19]:
def insert_grades(dic):
    for movie in dic:
        grades = get_grade(data[movie]['url'])
        data[movie]['관람객평점'] = grades[0]
        data[movie]['평론가평점'] = grades[1]
        data[movie]['네티즌평점'] = grades[2]
    return dic

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [26]:
def get_code(url):
    p = re.compile('[0-9]{6}')
    return p.findall(url)[0]

In [55]:
def get_reviews(url, page):
    '''
        url: 영화 url
        page: 수집하고자 하는 데이터 수 / 10
    '''
    code = get_code(url)
    reviews = []
    
    for p in range(1, page+1):
        url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'.format(code=code, page=p)
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        score_result = soup.find('div', {'class': 'score_result'})
        
        # 리스트
        lis = score_result.findAll('li')
        length = len(lis)
        
        for idx in range(length):
            dic = dict()
    
            # 평점
            try:
                dic['평점'] = lis[idx].find('em').text
            except:
                pass

            # 평점 작성자 닉네임 (괄호 제거 시 가산점)
            try:
                dic['닉네임'] = lis[idx].findAll('a')[0].find('span').text
            except:
                pass
            
            # 리뷰 본문
            try:
                dic['리뷰 본문'] = lis[idx].find('p').text[4:].strip()
            except:
                pass
            
            reviews.append(dic)

    return reviews

In [29]:
def insert_reviews(dic, data_num):
    '''
        data_num: 수집하려는 데이터 개수
    '''
    page = data_num//10
    for movie in dic:
        reviews = get_reviews(data[movie]['url'], page)
        data[movie]['리뷰'] = reviews
    return dic

### 4. 저장하기

In [56]:
import json

In [63]:
def dic_to_json(dic, filename):
    with open('./{}.json'.format(filename),'w', encoding='utf-8') as fp:
        json.dump(dic, fp, indent=4, ensure_ascii=False)

### 5. 크롤링하기

In [16]:
data = movie_title_url_actor(5, 3, data)

In [17]:
data

{'정직한 후보': {'url': '/movie/bi/mi/basic.nhn?code=186821',
  'actors': ['라미란', '김무열', '나문희']},
 '작은 아씨들': {'url': '/movie/bi/mi/basic.nhn?code=186613',
  'actors': ['시얼샤 로넌', '엠마 왓슨', '플로렌스 퓨']},
 '클로젯': {'url': '/movie/bi/mi/basic.nhn?code=181925',
  'actors': ['하정우', '김남길', '허율']},
 '기생충': {'url': '/movie/bi/mi/basic.nhn?code=161967',
  'actors': ['송강호', '이선균', '조여정']},
 '수퍼 소닉': {'url': '/movie/bi/mi/basic.nhn?code=172816',
  'actors': ['짐 캐리', '제임스 마스던', '벤 슈와츠']}}

In [20]:
data = insert_grades(data)

In [21]:
data

{'정직한 후보': {'url': '/movie/bi/mi/basic.nhn?code=186821',
  'actors': ['라미란', '김무열', '나문희'],
  '관람객평점': 9.11,
  '평론가평점': 5.38,
  '네티즌평점': 7.96},
 '작은 아씨들': {'url': '/movie/bi/mi/basic.nhn?code=186613',
  'actors': ['시얼샤 로넌', '엠마 왓슨', '플로렌스 퓨'],
  '관람객평점': 9.459999999999999,
  '평론가평점': 8.0,
  '네티즌평점': 8.98},
 '클로젯': {'url': '/movie/bi/mi/basic.nhn?code=181925',
  'actors': ['하정우', '김남길', '허율'],
  '관람객평점': 8.61,
  '평론가평점': 5.5,
  '네티즌평점': 6.98},
 '기생충': {'url': '/movie/bi/mi/basic.nhn?code=161967',
  'actors': ['송강호', '이선균', '조여정'],
  '관람객평점': 9.07,
  '평론가평점': 9.06,
  '네티즌평점': 8.49},
 '수퍼 소닉': {'url': '/movie/bi/mi/basic.nhn?code=172816',
  'actors': ['짐 캐리', '제임스 마스던', '벤 슈와츠'],
  '관람객평점': 8.66,
  '평론가평점': 5.0,
  '네티즌평점': 8.370000000000001}}

In [53]:
data = insert_reviews(data, 20)

10
김다솜(kmjd****)
너무 재밌었어용ㅠ 라미란 짱멋…
10
벚꽃의계절(twic****)
역시 라미란 코미디 연기는 진리다 ㅋㅋㅋㅋ
10
엥 댓글 알바라고 하먄서 평점 ㅈㄴ 낮춰놓는 게 더 댓글 알바같음 ;; 노잼 조폭 알탕 영화들보다 훨~~씬 나음 라미란 배우님 나문희 배우님 김무열 배우님 윤경호 배우님 외 많은 배우님들 ㅇ코믹 연기 넘 조와요 ㅈㄴ...
9
옹잉앙웅(t21g****)
개그 코드가 딱 내 스타일이야ㅋㅋ
6
lililli(wken****)
솔직히 그렇게 엄청 웃긴지는 모르겠어요
10
공주(prin****)
아니 사람들마다 좋아하는 요소가 다르고 생각하는게 다른데 왜 자기랑 다르면 다 댓글알바라하는거지?; 일반화 오지네
9
샤샤샷(pnxq****)
기대없이 봐서 그런가 너무 웃음-_-ㅋㅋ
10
아니진짜 개재밌음 ㅠㅠ 개웃김 미쳤음 ㅋㅋㅋ역대급 코미디임ㅋㅋㅋ 진짜 한 1년동안 재밌는 영화를 못봤는데 ㄹㅇ 개꿀잼 와... 사실 극한직업도 그렇게 웃기진 않다고 생각했는데 진짜 이 영화 스토리도 그렇고 연기도 그...
9
양다리(atdx****)
코미디영화인데 뭔가 코미디영화같기도하고 가족영화 같기도하고 재밌더라 ㅋ
10
김고은(safi****)
오락영화로 즐기기엔 손색없다
10
영(seyo****)
예고편보고도 웃겼는데ㅋㅋㅋ풀로보니까 더웃김ㅋㅋㅋㅋ 반차라서 조조로 봤는데 웃음소리가 마스크 뚫고 나감ㅋㅋㅋ
4
김호민(ooho****)
음... 내가 생각하는 재미랑 여기 사람들이 재밌다고 말하는 재미랑 다른건가..??
10
잇츠미나현(sue7****)
초반 부터 재밌고 유쾌 했습니다 ~ ㅎㅎㅎ 라미란 연기가 하드캐리
1
oen88(e2ro****)
대체 어떤부분이 웃낀거야 ??
5
솔직히 말할게요. 보고 왔는데 별로인 영화에요. 많이 오글거리기도 하고 전형적인 어설픈 코미디 한국영화의 클리셰가 다 들어가있네요.  저도 유튜브랑 여기 평점들 읽고 꽤 기대하면서, 그래도 괜찮은 영화겠지 했는데. ...
10
interest(park****)
너무 너무

In [54]:
data

{'정직한 후보': {'url': '/movie/bi/mi/basic.nhn?code=186821',
  'actors': ['라미란', '김무열', '나문희'],
  '관람객평점': 9.11,
  '평론가평점': 5.38,
  '네티즌평점': 7.96,
  '리뷰': [{'평점': '10', '닉네임': '김다솜(kmjd****)', '리뷰 본문': '너무 재밌었어용ㅠ 라미란 짱멋…'},
   {'평점': '10', '닉네임': '벚꽃의계절(twic****)', '리뷰 본문': '역시 라미란 코미디 연기는 진리다 ㅋㅋㅋㅋ'},
   {'평점': '10',
    '리뷰 본문': '엥 댓글 알바라고 하먄서 평점 ㅈㄴ 낮춰놓는 게 더 댓글 알바같음 ;; 노잼 조폭 알탕 영화들보다 훨~~씬 나음 라미란 배우님 나문희 배우님 김무열 배우님 윤경호 배우님 외 많은 배우님들 ㅇ코믹 연기 넘 조와요 ㅈㄴ...'},
   {'평점': '9', '닉네임': '옹잉앙웅(t21g****)', '리뷰 본문': '개그 코드가 딱 내 스타일이야ㅋㅋ'},
   {'평점': '6', '닉네임': 'lililli(wken****)', '리뷰 본문': '솔직히 그렇게 엄청 웃긴지는 모르겠어요'},
   {'평점': '10',
    '닉네임': '공주(prin****)',
    '리뷰 본문': '아니 사람들마다 좋아하는 요소가 다르고 생각하는게 다른데 왜 자기랑 다르면 다 댓글알바라하는거지?; 일반화 오지네'},
   {'평점': '9', '닉네임': '샤샤샷(pnxq****)', '리뷰 본문': '기대없이 봐서 그런가 너무 웃음-_-ㅋㅋ'},
   {'평점': '10',
    '리뷰 본문': '아니진짜 개재밌음 ㅠㅠ 개웃김 미쳤음 ㅋㅋㅋ역대급 코미디임ㅋㅋㅋ 진짜 한 1년동안 재밌는 영화를 못봤는데 ㄹㅇ 개꿀잼 와... 사실 극한직업도 그렇게 웃기진 않다고 생각했는데 진짜 이 영화 스토리도 그렇고 연기도 그...'},
   {'평점': '9',
    '닉네임': '양다리(atdx***

In [64]:
dic_to_json(data, 'assignment')